In [1]:
# !pip install matplotlib
# !pip install gurobipy
# !pip install torch_geometric

In [2]:
# !pip install scikit-learn

In [3]:
import gurobipy as gp
from gurobipy import GRB
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader  # Corrected import
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import global_mean_pool  # For pooling in the decoder
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import random
import matplotlib.pyplot as plt  # For plotting losses

In [4]:
from parse_QP import my_callback

In [5]:
# Read the problem
number = "0031"
grb_model = gp.read(f"QPLIB_{number}.lp")

# Solution storage
grb_model._feasible_solutions = []
grb_model._relaxation_solutions = []
grb_model.setParam("MIPGap", 0.05)
#model.setParam("NodeLimit", 100)  # Explore a limited number of nodes

# Optimize
grb_model.optimize(my_callback)

Restricted license - for non-production use only - expires 2026-11-23
Read LP format model from file QPLIB_0031.lp
Reading time = 0.00 seconds
obj: 32 rows, 60 columns, 120 nonzeros
Set parameter MIPGap to value 0.05
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel Xeon Processor (Icelake), instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
MIPGap  0.05

Optimize a model with 32 rows, 60 columns and 120 nonzeros
Model fingerprint: 0x00d24133
Model has 464 quadratic objective terms
Variable types: 30 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 3654.4800000
Presolve time: 0.00s
Presolved: 901 rows, 526 columns, 2321 nonzeros
Preso

In [6]:
# Retrieve optimal solution if available
if grb_model.status == GRB.OPTIMAL:
    optimal_solution = grb_model.getAttr('X', grb_model.getVars())
    print("Optimal solution:", optimal_solution)
else:
    print(f"Model status: {grb_model.status}")

Optimal solution: [0.0, 0.0, 0.3328307282391785, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1574644983911741, 0.25564451699144053, 0.0, 0.16105507040036132, 0.0, 0.0, 0.09300518597785103, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 1.0, 0.0, -0.0, -0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, -0.0, 0.0, 1.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]


In [7]:
## Extracting bounds

import gurobipy as gp

# Read the problem
number = "0031"
grb_model = gp.read(f"QPLIB_{number}.lp")

variable_bounds = {}
for var in grb_model.getVars():
    variable_bounds[var.VarName] = {'Lower': var.LB, 'Upper': var.UB}


Read LP format model from file QPLIB_0031.lp
Reading time = 0.00 seconds
obj: 32 rows, 60 columns, 120 nonzeros


In [8]:
## Extracting Q, A, b, d, etc

from parse_QP import parse_qplib_file

# Replace '0031' with the desired file number
data = parse_qplib_file('0031')

# Access the data
A = data['A']
b_vector = data['b_vector']
E = data['E']
d_vector = data['d']
Q = data['Q']
variables_info = data['variables_info']
variables_info = [v[0] for v in variables_info]
binary_indices = data['binary_indices']
variable_indices = data['variable_indices']

m, n = A.shape
# Get indices of non-zero elements in A
row_indices, col_indices = np.nonzero(A)
edge_weights = A[row_indices, col_indices]

# Map variable types to numerical values
# Node types: 0 - continuous, 1 - binary
variable_types = np.array([0 if v[0] == 'x' else 1 for v in variables_info])

# Collect indices of continuous and binary variables
continuous_indices = np.where(variable_types == 0)[0]
binary_indices = np.where(variable_types == 1)[0]
n_continuous = len(continuous_indices)
n_binary = len(binary_indices)


In [18]:
import os
import pickle
from generate_solutions import generate_feasible_solutions, generate_infeasible_solutions

# Set generate_new to False by default
generate_new = True
feasible_data_file = 'feasible_data.pkl'
infeasible_data_file = 'infeasible_data.pkl'

# Load or generate feasible solutions (as in your original code)
if not generate_new and os.path.exists(feasible_data_file):
    # Load feasible data
    with open(feasible_data_file, 'rb') as f:
        feasible_data = pickle.load(f)
    feasible_solutions = feasible_data['solutions']
    feasible_costs = feasible_data['costs']
    print("Loaded existing feasible solutions from file.")
else:
    # Generate feasible solutions
    num_objectives = 2000  # Adjust the number as needed
    feasible_solutions, feasible_costs = generate_feasible_solutions(
        A, E, Q, variables_info, b_vector, d_vector, num_objectives
    )
    # Save the generated data for future use
    feasible_data = {'solutions': feasible_solutions, 'costs': feasible_costs}
    with open(feasible_data_file, 'wb') as f:
        pickle.dump(feasible_data, f)
    print("Generated and saved feasible solutions.")

# Now, load or generate infeasible solutions
if not generate_new and os.path.exists(infeasible_data_file):
    # Load infeasible data
    with open(infeasible_data_file, 'rb') as f:
        infeasible_data = pickle.load(f)
    infeasible_solutions = infeasible_data['solutions']
    infeasible_costs = infeasible_data['costs']
    print("Loaded existing infeasible solutions from file.")
else:
    # Generate infeasible solutions
    num_infeasible_samples = 2000  # Adjust as needed
    infeasible_solutions, infeasible_costs = generate_infeasible_solutions(
        A, E, variables_info, b_vector, d_vector, Q, num_infeasible_samples, feasible_solutions
    )
    # Save the generated data for future use
    infeasible_data = {'solutions': infeasible_solutions, 'costs': infeasible_costs}
    with open(infeasible_data_file, 'wb') as f:
        pickle.dump(infeasible_data, f)
    print("Generated and saved infeasible solutions.")


Generating Feasible Solutions: 100%|██████████| 2000/2000 [03:48<00:00,  8.75it/s]


Generated and saved feasible solutions.
Generating infeasible samples...


Infeasible Samples: 100%|██████████| 2000/2000 [00:00<00:00, 2955.45it/s]

Generated and saved infeasible solutions.


In [19]:
import torch
import torch.nn.functional as F
from torch_geometric.data import DataLoader
import numpy as np
from networks import GNNModelObj

# Import the data preparation functions
from data_preparation_obj import (
    prepare_edge_index_and_attr,
    create_data_list,
    normalize_node_features,
    normalize_targets,
    split_data,
)

# Assuming feasible_solutions, feasible_costs, and Q are already defined

# Prepare edge information from Q
edge_index, edge_attr = prepare_edge_index_and_attr(Q)

# Create Data objects
data_list = create_data_list(feasible_solutions, feasible_costs, edge_index, edge_attr)

# Normalize node features
data_list, mean, std = normalize_node_features(data_list)

# Normalize target costs
data_list, target_mean, target_std = normalize_targets(data_list)

# Split into train and test sets
train_data, test_data = split_data(data_list, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

# Initialize model, optimizer, and loss function
model = GNNModelObj(hidden_channels=128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Training loop
def train():
    model.train()
    total_loss = 0
    total_original_loss = 0  # To accumulate loss in original cost values
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out.view(-1), data.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs

        # Compute loss in original cost values
        # De-normalize predictions and targets
        pred_cost = out.view(-1) * target_std + target_mean
        actual_cost = data.y.view(-1) * target_std + target_mean
        original_loss = criterion(pred_cost, actual_cost)
        total_original_loss += original_loss.item() * data.num_graphs

    average_loss = total_loss / len(train_data)
    average_original_loss = total_original_loss / len(train_data)
    return average_loss, average_original_loss

# Testing
def test(loader):
    model.eval()
    total_loss = 0
    total_original_loss = 0
    with torch.no_grad():
        for data in loader:
            out = model(data)
            loss = criterion(out.view(-1), data.y.view(-1))
            total_loss += loss.item() * data.num_graphs

            # Compute loss in original cost values
            pred_cost = out.view(-1) * target_std + target_mean
            actual_cost = data.y.view(-1) * target_std + target_mean
            original_loss = criterion(pred_cost, actual_cost)
            total_original_loss += original_loss.item() * data.num_graphs

    average_loss = total_loss / len(loader.dataset)
    average_original_loss = total_original_loss / len(loader.dataset)
    return average_loss, average_original_loss

# Training epochs
for epoch in range(1, 31):
    loss, original_loss = train()
    if epoch % 1 == 0:
        print(f'Epoch {epoch}, Normalized Loss: {loss:.4f}, Original Loss: {original_loss:.4f}')

# Testing
test_loss, test_original_loss = test(test_loader)
print(f'\nTest Normalized MSE Loss: {test_loss:.4f}, Test Original MSE Loss: {test_original_loss:.4f}')

# Function to display predictions
def show_predictions(data_list, loader_name="Training"):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for data in data_list:
            data = data.to('cpu')
            out = model(data)
            # De-normalize predictions and targets
            pred_cost = out.item() * target_std.item() + target_mean.item()
            actual_cost = data.y.item() * target_std.item() + target_mean.item()
            predictions.append(pred_cost)
            actuals.append(actual_cost)
    print(f"\n{loader_name} Set Predictions:")
    for i in range(len(predictions)):
        print(f"Sample {i+1}: Predicted Cost = {predictions[i]:.4f}, Actual Cost = {actuals[i]:.4f}")

# Show predictions for the training set
show_predictions(train_data, loader_name="Training")

# Show predictions for the test set
show_predictions(test_data, loader_name="Test")


/opt/conda/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Normalized Loss: 0.9302, Original Loss: 0.1248
Epoch 2, Normalized Loss: 0.6363, Original Loss: 0.0854
Epoch 3, Normalized Loss: 0.2514, Original Loss: 0.0337
Epoch 4, Normalized Loss: 0.1262, Original Loss: 0.0169
Epoch 5, Normalized Loss: 0.0486, Original Loss: 0.0065
Epoch 6, Normalized Loss: 0.0227, Original Loss: 0.0031
Epoch 7, Normalized Loss: 0.0113, Original Loss: 0.0015
Epoch 8, Normalized Loss: 0.0069, Original Loss: 0.0009
Epoch 9, Normalized Loss: 0.0052, Original Loss: 0.0007
Epoch 10, Normalized Loss: 0.0036, Original Loss: 0.0005
Epoch 11, Normalized Loss: 0.0041, Original Loss: 0.0006
Epoch 12, Normalized Loss: 0.0023, Original Loss: 0.0003
Epoch 13, Normalized Loss: 0.0030, Original Loss: 0.0004
Epoch 14, Normalized Loss: 0.0030, Original Loss: 0.0004
Epoch 15, Normalized Loss: 0.0029, Original Loss: 0.0004
Epoch 16, Normalized Loss: 0.0025, Original Loss: 0.0003
Epoch 17, Normalized Loss: 0.0076, Original Loss: 0.0010
Epoch 18, Normalized Loss: 0.0095, Origi

In [21]:
from networks import GNNModelConstraints

# Import the data preparation functions
from data_preparation_constraints import (
    prepare_constraint_edge_data,
    create_constraint_data_list,
    normalize_constraint_variable_features,
    normalize_constraint_targets,
)

# Get dimensions
m, n = A.shape           # m inequality constraints, n variables
p = E.shape[0]           # p equality constraints

# Prepare edge information from A and E
edge_index, edge_attr = prepare_constraint_edge_data(A, E, n, m, p)

# Create Data objects
data_list = create_constraint_data_list(
    feasible_solutions, A, E, b_vector, d_vector, edge_index, edge_attr, n, m, p
)

# Normalize variable node features
data_list, mean_var, std_var = normalize_constraint_variable_features(data_list, n)

# No need to normalize constraint node features (b_vector and d_vector) as they are constants

# Normalize target y
data_list, target_mean, target_std = normalize_constraint_targets(data_list)

# Split into train and test sets
train_data, test_data = split_data(data_list, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

# Initialize model, optimizer, and loss function
model = GNNModelConstraints(hidden_channels=128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Training loop
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        # Collect constraint node indices (inequality and equality constraints)
        batch_size = data.num_graphs
        num_nodes_per_graph = n + m + p
        constraint_indices = []
        for i in range(batch_size):
            graph_node_offset = i * num_nodes_per_graph
            # Inequality constraint nodes
            inequality_constraint_node_indices = torch.arange(
                graph_node_offset + n,
                graph_node_offset + n + m
            )
            # Equality constraint nodes
            equality_constraint_node_indices = torch.arange(
                graph_node_offset + n + m,
                graph_node_offset + n + m + p
            )
            constraint_indices.append(torch.cat([
                inequality_constraint_node_indices,
                equality_constraint_node_indices
            ]))
        constraint_indices = torch.cat(constraint_indices)
        pred_y = out[constraint_indices].view(-1)
        target_y = data.y.view(-1)
        loss = criterion(pred_y, target_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs

    average_loss = total_loss / len(train_data)
    return average_loss

# Testing function
def test(loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            out = model(data)
            # Collect constraint node indices (inequality and equality constraints)
            batch_size = data.num_graphs
            num_nodes_per_graph = n + m + p
            constraint_indices = []
            for i in range(batch_size):
                graph_node_offset = i * num_nodes_per_graph
                # Inequality constraint nodes
                inequality_constraint_node_indices = torch.arange(
                    graph_node_offset + n,
                    graph_node_offset + n + m
                )
                # Equality constraint nodes
                equality_constraint_node_indices = torch.arange(
                    graph_node_offset + n + m,
                    graph_node_offset + n + m + p
                )
                constraint_indices.append(torch.cat([
                    inequality_constraint_node_indices,
                    equality_constraint_node_indices
                ]))
            constraint_indices = torch.cat(constraint_indices)
            pred_y = out[constraint_indices].view(-1)
            target_y = data.y.view(-1)
            loss = criterion(pred_y, target_y)
            total_loss += loss.item() * data.num_graphs

    average_loss = total_loss / len(loader.dataset)
    return average_loss

# Training epochs
for epoch in range(1, 31):
    loss = train()
    if epoch % 1 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')

# Testing
test_loss = test(test_loader)
print(f'\nTest MSE Loss: {test_loss:.4f}')

import pandas as pd
from IPython.display import display

def show_constraint_predictions(model, data_list, n, m, p, target_mean, target_std, loader_name="Training"):
    model.eval()
    print(f"\n{loader_name} Set Predictions:")
    with torch.no_grad():
        # Only display results for the first two samples
        for idx, data in enumerate(data_list[:2]):
            out = model(data)
            # Get indices of constraint nodes (node indices start from 0 in each graph)
            inequality_constraint_indices = torch.arange(n, n + m)
            equality_constraint_indices = torch.arange(n + m, n + m + p)
            # Predicted violations
            pred_inequality = out[inequality_constraint_indices].view(-1)
            pred_equality = out[equality_constraint_indices].view(-1)
            # Actual violations
            actual_inequality = data.y[:m]
            actual_equality = data.y[m:]
            # De-normalize predictions and actual values
            pred_inequality = pred_inequality * target_std + target_mean
            pred_equality = pred_equality * target_std + target_mean
            actual_inequality = actual_inequality * target_std + target_mean
            actual_equality = actual_equality * target_std + target_mean
            # Prepare data for DataFrame
            print(f"\nSample {idx+1}:")
            # Inequality Constraints DataFrame
            inequality_data = {
                'Constraint': [f'Inequality {j+1}' for j in range(m)],
                'Actual Violation': actual_inequality.cpu().numpy(),
                'Predicted Violation': pred_inequality.cpu().numpy()
            }
            inequality_df = pd.DataFrame(inequality_data)
            print("Inequality Constraints:")
            display(inequality_df)
            # Equality Constraints DataFrame
            if p > 0:
                equality_data = {
                    'Constraint': [f'Equality {j+1}' for j in range(p)],
                    'Actual Violation': actual_equality.cpu().numpy(),
                    'Predicted Violation': pred_equality.cpu().numpy()
                }
                equality_df = pd.DataFrame(equality_data)
                print("Equality Constraints:")
                display(equality_df)



# Show predictions for the training set
show_constraint_predictions(model, train_data, n, m, p, target_mean, target_std, loader_name="Training")

# Show predictions for the test set
show_constraint_predictions(model, test_data, n, m, p, target_mean, target_std, loader_name="Test")


/opt/conda/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.3600
Epoch 2, Loss: 0.1558
Epoch 3, Loss: 0.1382
Epoch 4, Loss: 0.1267
Epoch 5, Loss: 0.1198
Epoch 6, Loss: 0.1022
Epoch 7, Loss: 0.0710
Epoch 8, Loss: 0.0445
Epoch 9, Loss: 0.0381
Epoch 10, Loss: 0.0365
Epoch 11, Loss: 0.0361
Epoch 12, Loss: 0.0353
Epoch 13, Loss: 0.0345
Epoch 14, Loss: 0.0344
Epoch 15, Loss: 0.0342
Epoch 16, Loss: 0.0334
Epoch 17, Loss: 0.0331
Epoch 18, Loss: 0.0327
Epoch 19, Loss: 0.0327
Epoch 20, Loss: 0.0343
Epoch 21, Loss: 0.0344
Epoch 22, Loss: 0.0324
Epoch 23, Loss: 0.0321
Epoch 24, Loss: 0.0320
Epoch 25, Loss: 0.0319
Epoch 26, Loss: 0.0318
Epoch 27, Loss: 0.0318
Epoch 28, Loss: 0.0316
Epoch 29, Loss: 0.0317
Epoch 30, Loss: 0.0315

Test MSE Loss: 0.0261

Training Set Predictions:

Sample 1:
Inequality Constraints:


,Constraint,Actual Violation,Predicted Violation
0,Inequality 1,-2.000000,-1.977009
1,Inequality 2,0.000000,0.001178
2,Inequality 3,0.000000,0.001178
3,Inequality 4,0.000000,0.001178
4,Inequality 5,0.000000,0.001178
5,Inequality 6,0.000000,0.001178
6,Inequality 7,0.000000,0.001178
7,Inequality 8,0.000000,0.001178
8,Inequality 9,-0.605719,-0.612362
9,Inequality 10,0.000000,0.001178


Equality Constraints:


,Constraint,Actual Violation,Predicted Violation
0,Equality 1,0.0,0.084905


...

,Constraint,Actual Violation,Predicted Violation
0,Equality 1,0.0,-0.187934



Sample 2:
Inequality Constraints:


,Constraint,Actual Violation,Predicted Violation
0,Inequality 1,-3.000000,-2.977219
1,Inequality 2,0.000000,0.001178
2,Inequality 3,0.000000,0.001178
3,Inequality 4,0.000000,0.001178
4,Inequality 5,0.000000,0.001178
5,Inequality 6,0.000000,0.001178
6,Inequality 7,0.000000,0.001178
7,Inequality 8,0.000000,0.001178
8,Inequality 9,0.000000,0.001178
9,Inequality 10,0.000000,0.001178


Equality Constraints:


,Constraint,Actual Violation,Predicted Violation
0,Equality 1,0.0,-0.013974


In [ ]:
show_predictions(test_data, loader_name="Test")